In [70]:
import pandas as pd

In [71]:
rsp_df = pd.read_csv("../Data/processed/dataset_2016-19")
pt = pd.read_csv("../Data/misc/poverty_thresholds.csv")

C:\Users\Patrick\AppData\Local\Temp\ipykernel_17316\2492198880.py:1: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  rsp_df = pd.read_csv("../Data/processed/dataset_2016-19")


In [72]:
# Create target variable


def classify_resettlement(x):
    """Classifies if resettlement was fully successful or not"""

    # CRITERIA:
    # Subject is either employed or in education / training, unless they are not eligible
    # Subject makes an income above the poverty level, if employed
    # Subject is either a permanent legal resident or is planning to adjust status

    # Gets poverty threshold data for year
    thresholds = pt[pt["Year"] == x["survey_year"]].iloc[0].tolist()

    # Check that subject is not unemployed
    if x["ui_emprate"] in ["unemployed", "don't know and/or refused"]:
        return 0

    # If employed, check that target makes wage above poverty threshold for given household size
    if x["ui_emprate"] == "employed":
        try:
            if (
                float(x["ui_qn8a_annual"]) + float(x["ui_qn10a_annual"])
                < thresholds[int(x["numppl"] - 1)]
            ):
                return 0
        except ValueError:
            return 0

    # Check subject's residency status
    if x["ui_lpr"] not in [
        "already adjusted lpr status",
        "plans to adjust lpr status in future",
    ]:
        return 0

    # Check for people who are not in labor force
    # People who gave up job search due to:
    # limited english
    # Unable to find work
    if x["qn17_01"] == "limited english":
       return 0
    if x["qn17_06"] == "tried to find work but couldn't":
       return 0

    return 1


rsp_df["t_resettlement"] = rsp_df.apply(lambda x: classify_resettlement(x), axis=1)

In [73]:
# Show breakdown of target variable counts
rsp_df["t_resettlement"].value_counts()

t_resettlement
1    3996
0    1772
Name: count, dtype: int64

In [74]:
# Write output

rsp_df.to_csv("../Data/processed/dataset_2016-19_target", index=False)